# Heart rate analysis

This tutorial shows how to extract heart rate estimates using photoplethysmography (PPG) data and accelerometer data. The pipeline consists of a stepwise approach ot determine signal quality, assessing both PPG morphology and accounting for periodic artifacts using the accelerometer. Based on the signal quality, we extract high-quality segments and estimate the heart rate for every 2 s using the smoothed pseudo Wigner-Ville Distribution. 

## Load example data

Example PPG + accelerometer (from IMU files) data from a participant of the Personalized Parkinson Project is loaded. The data was prepared as explained in `data_preparation.ipynb`. The prepared IMU data contains both accelerometer and gyroscope data, but only accelerometer is used in this pipeline alongside the PPG data. First, we scan the metadata of the PPG and IMU to see if there is overlapping data using scan_and_sync_segments function, whereafter we load the corresponding dataframes.

In [ ]:
from pathlib import Path
from paradigma.util import load_tsdf_dataframe
from paradigma.config import PPGConfig, IMUConfig
from paradigma.preprocessing import scan_and_sync_segments
import tsdf

path_to_data = Path('../../tests/data')
path_to_prepared_data = path_to_data / '1.prepared_data'

ppg_config = PPGConfig()
imu_config = IMUConfig()
# First, we scan and synchronize the data based on the metadatas
metadatas_ppg, metadatas_imu = scan_and_sync_segments(path_to_prepared_data / 'ppg', path_to_prepared_data / 'imu')
# Then, we load the data
metadata_time_ppg = metadatas_ppg[0][ppg_config.time_filename]
metadata_values_ppg = metadatas_ppg[0][ppg_config.values_filename]
df_ppg = tsdf.load_dataframe_from_binaries([metadata_time_ppg, metadata_values_ppg], tsdf.constants.ConcatenationType.columns)

metadata_time_imu = metadatas_imu[0][imu_config.time_filename]
metadata_values_imu = metadatas_imu[0][imu_config.values_filename]
df_imu = tsdf.load_dataframe_from_binaries([metadata_time_imu, metadata_values_imu], tsdf.constants.ConcatenationType.columns)

df_ppg, df_imu

(            time   green
 0        0.00000  649511
 1        0.00996  648214
 2        0.01992  646786
 3        0.02988  645334
 4        0.03984  644317
 ...          ...     ...
 64770  644.54720  553884
 64771  644.55716  554088
 64772  644.56712  554240
 64773  644.57708  555134
 64774  644.58704  557205
 
 [64775 rows x 2 columns],
             time  accelerometer_x  ...  gyroscope_y  gyroscope_z
 0        0.00000         0.550718  ...    32.012195   -26.097561
 1        0.01004         0.535885  ...    34.634146   -24.695122
 2        0.02008         0.504306  ...    36.768293   -22.926829
 3        0.03012         0.488517  ...    38.719512   -21.158537
 4        0.04016         0.494258  ...    41.280488   -20.304878
 ...          ...              ...  ...          ...          ...
 72942  730.74468         0.234928  ...    -2.256098     2.256098
 72943  730.75472         0.245455  ...    -1.707317     1.768293
 72944  730.76476         0.243541  ...    -1.585366     1.890244

## Step 1: Preprocess data

The first step after loading the data is to preprocess the data. Preprocessing starts by extracting the data where there is both PPG and IMU data. In this way, we discard the the first or last part of the segment when e.g. the PPG segment is recorded longer than the accelerometer. After this step, the preprocess_ppg_data function  resamples the values of both PPG and accelerometer data using uniformly distributed timestamps, since the sampling rate of both sensors is fixed but not uniform. After this, a bandpass filter (butterworth, 4th-order, cut-off frequencies: [0.4, 3.5]) is applied to the PPG signal and a high-pass (butterworth, 4th-order, cut-off: 0.2 Hz) filter is applied to the accelerometer. 

In [ ]:
from paradigma.preprocessing import preprocess_ppg_data
df_ppg_proc, df_acc_proc = preprocess_ppg_data(df_ppg, df_imu, ppg_config, imu_config)
df_ppg_proc, df_acc_proc

Shape of the original data: (64775, 2) (72947, 4)
Shape of the overlapping segments: (64775, 2) (64361, 4)


(             time        green
 0        0.000000 -1434.856838
 1        0.033333 -3461.717789
 2        0.066667 -5172.913796
 3        0.100000 -6343.203160
 4        0.133333 -6875.904054
 ...           ...          ...
 19333  644.433333 -2403.547097
 19334  644.466667 -7434.311944
 19335  644.500000 -8214.847078
 19336  644.533333 -5194.393329
 19337  644.566667    70.147000
 
 [19338 rows x 2 columns],
          time  accelerometer_x  ...  accelerometer_z  grav_accelerometer_z
 0        0.00         0.053078  ...        -0.273154             -0.000530
 1        0.01         0.038337  ...        -0.256899              0.002305
 2        0.02         0.006824  ...        -0.256739              0.005122
 3        0.03        -0.009156  ...        -0.273280              0.007919
 4        0.04        -0.003770  ...        -0.289007              0.010696
 ...       ...              ...  ...              ...                   ...
 64454  644.54         0.104185  ...        -0.029342  

## Step 2: Extract signal quality features

The preprocessed data (PPG & accelerometer) is windowed into overlapping windows of length `ppg_config.window_length_s` with a window step of `ppg_config.window_step_length_s`. From the PPG windows 10 time- and frequency domain features are extracted to assess PPG morphology and from the accelerometer windows one relative power feature is calculated to assess periodic motion artifacts. 

In [ ]:
from paradigma.config import SignalQualityFeatureExtractionConfig, SignalQualityFeatureExtractionAccConfig
from paradigma.heart_rate.heart_rate_analysis import extract_signal_quality_features

ppg_config = SignalQualityFeatureExtractionConfig()
acc_config = SignalQualityFeatureExtractionAccConfig()
df_features = extract_signal_quality_features(ppg_config, df_ppg_proc, acc_config, df_acc_proc)
df_features


,time,var,mean,median,kurtosis,skewness,signal_to_noise,auto_corr,f_dom,rel_power,spectral_entropy,acc_power_ratio
0,0.0,1.697510e+07,3418.696392,2892.815105,2.540447,0.277237,3.241634,0.282452,1.171875,0.179922,0.561186,0.014196
1,1.0,1.635033e+07,3350.769158,2892.815105,2.647951,0.420163,3.202313,0.276526,1.171875,0.201972,0.547812,0.015343
2,2.0,1.921248e+07,3698.963099,3310.773556,2.333687,0.481974,3.521882,0.210984,0.820312,0.264922,0.509015,0.042583
3,3.0,1.646562e+07,3389.325775,2946.588526,2.608187,0.733913,3.346236,0.225823,0.703125,0.270707,0.496051,0.034215
4,4.0,1.603135e+07,3364.366313,2912.280798,2.268768,0.440357,3.314565,0.305998,0.820312,0.264097,0.496697,0.056204
...,...,...,...,...,...,...,...,...,...,...,...,...
634,634.0,2.281660e+06,1106.006037,838.307729,4.058037,0.674184,2.072660,0.172052,0.703125,0.204028,0.577570,0.060486
635,635.0,2.224441e+06,1113.194933,838.307729,3.684159,0.067979,2.118019,0.236408,0.703125,0.221044,0.560362,0.053120
636,636.0,5.103141e+06,1627.618918,1030.113303,3.913068,0.852331,2.040094,0.235822,0.585938,0.157667,0.514717,0.058853
637,637.0,7.428728e+06,2093.736007,1520.640067,3.188753,0.305780,2.457446,0.322034,0.468750,0.150582,0.458477,0.061269


## Step 3: Signal quality classification

A trained logistic classifier is used to classify PPG signal quality and returns the `pred_sqa_proba`, which is the posterior probability of a PPG window to look like the typical PPG morphology (higher probability indicates toward the typical PPG morphology). The relative power feature from the accelerometer is compared to a threshold for periodic artifacts and therefore `pred_sqa_acc_label` returns a label indicating probably periodic motion artifacts (label 0) or no periodic motion artifacts (label 1). 

In [ ]:
from paradigma.config import SignalQualityClassificationConfig
from paradigma.heart_rate.heart_rate_analysis import signal_quality_classification

path_to_classifier = Path('../../tests/data/0.Classification/ppg')
config = SignalQualityClassificationConfig()
df_sqa = signal_quality_classification(df_features, config, path_to_classifier)
df_sqa

,pred_sqa_proba,pred_sqa_acc_label
0,0.006031,1
1,0.011725,1
2,0.068063,1
3,0.080808,1
4,0.074591,1
...,...,...
634,0.001387,1
635,0.001570,1
636,0.000380,1
637,0.000406,1


## Step 4: Heart rate estimation

For heart rate estimation, we extract segments of `config.tfd_length`. Using a 2-step approach, we calculate the smoothed-pseudo Wigner-Ville Distribution to obtain the frequency content of the PPG signal over time. For every 2-second window, we identified the frequency with the highest power for each data point and assigned the average of these frequency as the heart rate in that 2-second window. 

In [ ]:
from paradigma.config import HeartRateExtractionConfig
from paradigma.heart_rate.heart_rate_analysis import estimate_heart_rate

config = HeartRateExtractionConfig()
df_hr = estimate_heart_rate(df_sqa, df_ppg_proc, config)
df_hr

,rel_time,heart_rate
0,56.0,86.404715
1,58.0,86.640472
2,60.0,86.345776
3,62.0,84.872299
4,64.0,84.872299
5,66.0,84.194499


## Step 5: Heart rate aggregation

The final step is to aggregate all 2-second heart rate estimates. In the current example, the mode and 99th percentile are calculated. The mode represent the resting heart rate while the 99th percentile indicates the maximum heart rate. In Parkinson's disease, we expect that these two measures could reflect autonomic (dys)functioning.

In [ ]:
import pprint
from paradigma.heart_rate.heart_rate_analysis import aggregate_heart_rate
hr_values = df_hr['heart_rate'].values
df_hr_agg = aggregate_heart_rate(hr_values, aggregates = ['mode', '99p'])

pprint.pprint(df_hr_agg)

{'hr_aggregates': {'99p_heart_rate': 86.62868369351669,
                   'mode_heart_rate': 84.8722986247544},
 'metadata': {'nr_hr_est': 6}}
